# LB04: Ансамбли моделей

<div style="text-align: right"> ПСА 5 (Введение в машинное обучение). Мехмат, БГУ</div>
    
<div style="text-align: right"> Тишуров Алексей, 2021 </div>

Данный материал использует лицензию [Creative Commons CC BY-NC-SA 4.0.](https://creativecommons.org/licenses/by-nc-sa/4.0/) со всеми вытекающими. На прилагаемые к материалу датасеты лицензия не распространяется. 

В рамках данной лабораторной работы вы используете на практике два вида ансамблей: случайный лес (в реализации sklearn) и градиентный бустинг (в реализации lightgbm).

Для установки lightgbm для linux точно будет достаточно conda install -c conda-forge lightgbm.
На windows тоже должно сработать, но если будут проблемы, то может потребоваться VS.


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# Часть 1. Случайный лес

В первой часте вы будете работать с датасетами из прошлых лаб (предсказание оттока у мобильного оператора и предсказание цены автомобиля). Необходимо использовать все наработки из прошлых лаб, пользовать кросс-валидацией, вычислять те же метрики. Для задачи с предсказанием цены автомобиля не забыть про возможность (если она нужна) преобразования целевой переменной.

Таким образом, изучать датасет, его особенности и инженерить признаки снова не нужно, что ускоряет эту часть лабы.

## Задание 1.1. Обучение случайного леса

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_predict, cross_val_score, \
    GridSearchCV, train_test_split, KFold
from eli5.sklearn import explain_rf_feature_importance

Используйте наработки прошлой лабы (№3) и:
1. Обучите на датасете про предсказания цены автомобиля RandomForestRegressor и проведите анализ качества работы вашей модели. 
2. Сравните с качеством работы отдельного дерева.
3. Проанализируйте важность признаков с помощью feature_importances_
4. Проведите оптимизацию гиперпараметров с помощью GridSearchCV, добавив новые параметры леса к параметрам одного дерева из прошлой лабы. Используйте для этого не очень большое количество деревьев в лесу. После увеличьте его в несколько раз с лучшими гиперпараметрами и сравните изменение качества

## Loading and preprocess

In [3]:
train = pd.read_csv('train-data.csv', index_col=0)

In [4]:
train['Mileage'] = train['Mileage'].apply(lambda x: str(x).split()[0])
train['Power'] = train['Power'].apply(lambda x: str(x).split()[0])
train['Engine'] = train['Engine'].apply(lambda x: str(x).split()[0])
train['New_Price'] = train['New_Price'].apply(lambda x: str(x).split()).apply(lambda x: 
                                str(float(x[0])*100) if len(x) == 2 and x[1] == 'Cr' else x[0])

train.fillna(value=0, inplace=True)
train = train.replace('null', 0)

train['Mileage'] = train['Mileage'].apply(lambda x: np.nan_to_num(float(x)))
train['Power'] = train['Power'].apply(lambda x: np.nan_to_num(float(x)))
train['Engine'] = train['Engine'].apply(lambda x: np.nan_to_num(float(x)))
train['New_Price'] = train['New_Price'].apply(lambda x: np.nan_to_num(float(x)))

train['Transmission'] = train['Transmission'].apply(lambda x: 1 if x == 'Automatic' else 0)

In [5]:
categorical_cols = ['Location', 'Fuel_Type', 'Owner_Type']

train_cat = pd.get_dummies(train[categorical_cols])
train = pd.concat([train.drop(categorical_cols, axis=1), 
                    train_cat], axis=1)
train.drop('Name', axis=1, inplace=True)

transmission = train['Transmission']
train.drop('Transmission', axis=1, inplace=True)
train.insert(8, 'Transmission', transmission)

In [6]:
Y = train['Price'].copy(deep=True).to_numpy()
train.drop(['Price'], axis=1, inplace=True)

## Features transform

In [7]:
scaler = StandardScaler()
real_features = scaler.fit_transform(train.iloc[:, :7])
rest_features = train.iloc[:, 7:].to_numpy()

X = np.copy(np.hstack([real_features, rest_features]))

In [8]:
pt = PowerTransformer()
y = pt.fit_transform(Y.reshape(-1, 1)).flatten()

## Cross-validation initializing

In [9]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

## RandomForestRegressor initializing and fitting

In [10]:
reg = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)

In [11]:
y_pred = cross_val_predict(reg, X, y, cv=cv)
y_pred = pt.inverse_transform(y_pred.reshape(-1, 1)).flatten()

In [12]:
mean_squared_error(Y, y_pred), mean_absolute_error(Y, y_pred)

(16.87725132672729, 1.5123879265281241)

#### При количестве деревьев 100 и 120 качество сильно не меняется. Так как качество алгоритма близится к определенной асимптоте, можно остановиться на количестве деревьев 100. 

In [13]:
reg_tree = DecisionTreeRegressor(random_state=0)

In [14]:
y_pred = cross_val_predict(reg_tree, X, y, cv=cv)
y_pred = pt.inverse_transform(y_pred.reshape(-1, 1)).flatten()

In [15]:
mean_squared_error(Y, y_pred), mean_absolute_error(Y, y_pred)

(23.227966337027578, 1.9482722384149234)

#### Как можно видеть, качество ансамбля деревьев несколько лучше качества одного дерева.

## Feature importance

In [16]:
reg.fit(X, y)

RandomForestRegressor(n_jobs=-1, random_state=0)

In [17]:
explain_rf_feature_importance(reg, feature_names=list(train.columns))

Weight,Feature
0.5970 ± 0.0260,Power
0.2595 ± 0.0153,Year
0.0484 ± 0.0208,Engine
0.0206 ± 0.0070,Mileage
0.0198 ± 0.0052,Kilometers_Driven
0.0091 ± 0.0064,Transmission
0.0061 ± 0.0017,Location_Kolkata
0.0060 ± 0.0048,Seats
0.0060 ± 0.0034,New_Price
0.0036 ± 0.0016,Location_Hyderabad


## Hyperparameters selection

In [18]:
params = {
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 8, 10],
    'max_features': [1/3, 1/4, 'auto']
}

In [19]:
grid_reg = GridSearchCV(RandomForestRegressor(n_estimators=100, random_state=0), param_grid=params, 
                                                cv=cv, scoring='neg_mean_squared_error', verbose=True, n_jobs=-1)

In [20]:
%%time
grid_reg.fit(X, y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Wall time: 31.7 s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestRegressor(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [10, 15, 20, None],
                         'max_features': [0.3333333333333333, 0.25, 'auto'],
                         'min_samples_split': [2, 8, 10]},
             scoring='neg_mean_squared_error', verbose=True)

In [21]:
grid_reg.best_estimator_

RandomForestRegressor(max_depth=20, max_features=0.3333333333333333,
                      random_state=0)

In [22]:
best_reg = RandomForestRegressor(n_estimators=120, max_features=1/3,
                                max_depth=20, random_state=0, n_jobs=-1)

In [23]:
y_pred = cross_val_predict(best_reg, X, y, cv=cv)
y_pred = pt.inverse_transform(y_pred.reshape(-1, 1)).flatten()

In [24]:
mean_squared_error(Y, y_pred), mean_absolute_error(Y, y_pred)

(16.80806608493037, 1.5166557705908172)

In [25]:
explain_rf_feature_importance(grid_reg.best_estimator_, feature_names=list(train.columns))

Weight,Feature
0.3232 ± 0.3942,Power
0.1901 ± 0.3558,Engine
0.1836 ± 0.0876,Year
0.0945 ± 0.2495,Transmission
0.0435 ± 0.0338,Kilometers_Driven
0.0375 ± 0.0446,Mileage
0.0344 ± 0.1191,Fuel_Type_Diesel
0.0219 ± 0.0888,Fuel_Type_Petrol
0.0159 ± 0.0362,New_Price
0.0105 ± 0.0175,Seats


#### Немного получилось улучшить качество с перебором гиперпараметров.

## Задание 1.2. Использование рандомизированного поиска гиперпараметров

In [26]:
from sklearn.model_selection import RandomizedSearchCV

Использование GridSearch может привести к тому, что придется перебирать слишком много комбинаций параметров, так как их общее количество равно произведению уникальных значений каждого из перебираемых параметров. Поэтому, часто имеет смысл использовать другие варианты перебора. И самым простым из них является случайный поиск (RandomizedSearchCV).

В этом случае вместо сетки (param_grid) вам нужно указать распределения или конкретные списки значений требуемых параметров, откуда будут случайным образом выбираться значения. Длительность поиска контролируется с помощью параметра n_iter (например, 250). На практике оказывается, что случайный перебор параметров дает хорошие результаты за намного более короткое время.


Основные параметры случайного леса дискретные, поэтому вам предлагается указывать распределения в види списков значений, а не распределений из scipy(как это в некоторых случаях делается в документации для линейных моделей с параметрами С и l1_ratio, а вам, возможно, захочется делать для некоторых параметров бустинга). Если параметров много, то удобно воспользоваться range:

param_distr = {'max_depth': list(range(3, 12)) + [None]),
               'min_samples_split': range(2, 200, 5)}
               
               
Сравните время на перебор разумного числа параметров в рандомизированном поиске и полного перебора по сетке. Сравните итоговые параметры и качество моделей.

In [142]:
param_distr = {
    'max_depth': list(range(3, 20)) + [None],
    'min_samples_split': range(2, 200, 5),
    'max_features': [1/3, 1/4, 'auto']
}

In [147]:
rand_reg = RandomizedSearchCV(RandomForestRegressor(n_estimators=100, random_state=0),
                            param_distr, n_iter=180, cv=cv, scoring='neg_mean_squared_error',
                            verbose=True, random_state=0)

In [148]:
%%time
rand_reg.fit(X, y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
Wall time: 12min 54s


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestRegressor(random_state=0), n_iter=180,
                   param_distributions={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12, 13, 14, 15, 16,
                                                      17, 18, 19, None],
                                        'max_features': [0.3333333333333333,
                                                         0.25, 'auto'],
                                        'min_samples_split': range(2, 200, 5)},
                   random_state=0, scoring='neg_mean_squared_error',
                   verbose=True)

In [149]:
rand_reg.best_estimator_

RandomForestRegressor(max_depth=18, min_samples_split=7, random_state=0)

In [150]:
reg = RandomForestRegressor(n_estimators=100, max_depth=18, min_samples_split=7,
                            n_jobs=-1, random_state=0)

In [151]:
y_pred = cross_val_predict(best_reg, X, y, cv=cv)
y_pred = pt.inverse_transform(y_pred.reshape(-1, 1)).flatten()

In [152]:
mean_squared_error(Y, y_pred), mean_absolute_error(Y, y_pred)

(16.808066084930367, 1.516655770590817)

#### Времени на случайный поиск по сетке потребовалось больше конкретно в данном случае, зато мы смогли задать больше параметров (то есть потенциально мы бы смогли еще улучшить качество модели), но, если бы мы сделали полный перебор по сетке с такими же параметрами, времени бы затратилось куда больше.

## Задание 1.3 (На 9). Упрощенная реализация случайного леса

In [27]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, \
    recall_score, f1_score, roc_auc_score

В этом задании вам предлагается написать упрощенную реализацию случайного леса для задачи классификации. Для этого мы будем считать, что дерево у нас уже есть.


Необходимо определить методы fit, predict_proba и predict для заготовки класса ниже. Необходимо сделать следующее:
1. Реализовать бутстреп и метод подпространств для получения выборки для каждого нового дерева в ансамбле. Для этого рекомендуется использовать np.random.choice с повторениями и без повторений. 
2. На каждом таком датасете обучать решающее дерево для простоты передавая ему только параметры max_depth,  min_samples_split и criterion. Остальные параметры оставить по умолчанию
3. Повторять этот процесс в цикле количество раз, указанное в n_estimamtors. Все деревья добавлять в list _trees
4. Для предсказания вероятностей в цикле предсказать вероятности каждым деревом и усреднить. Для предсказания класса передавать дополнительно порог th. Если он None, то должен использоваться порог 0.5


После сравните вашу реализацию и RandomForestClassifier с выбранными параметрами


In [28]:
class SimpleRandomForestClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, n_estimators=100,
                 criterion='gini',
                 max_features='auto',
                 max_depth=None,
                 min_samples_split=2):
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
        self._trees = []
        
    
    def fit(self, X, y):
        for _ in range(self.n_estimators):
            # Bootstrap.  
            bootstrap_indices = np.random.choice(X.shape[0], X.shape[0])
            X_train = X[bootstrap_indices]
            y_train = y[bootstrap_indices]

            clf_tree = DecisionTreeClassifier(criterion=self.criterion, max_depth=self.max_depth, max_features=self.max_features,
                                                min_samples_split=self.min_samples_split, random_state=42)
            
            clf_tree.fit(X_train, y_train)

            self._trees.append(clf_tree)

        return self

    def predict_proba(self, X):
        y_pred = [clf.predict_proba(X) for clf in self._trees]
        
        return np.mean(y_pred, axis=0)

    def predict(self, X, th=None):
        if th is None:
            th = 0.5

        return (self.predict_proba(X)[:, 1] > th).astype(int)

In [29]:
df = pd.read_csv('dataset.csv')

In [30]:
df.head()

,CustomerID,Gender,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,...,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,7590-VHVEG,0,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,1,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,1,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,1,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,0,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [31]:
X_df, y = \
df.loc[:, (df.columns != 'Churn') & (df.columns != 'CustomerID')].copy(deep=True), df['Churn'].copy(deep=True)

In [32]:
for col_name in X_df.columns:
    col_values = X_df[col_name].value_counts()
    if set(col_values.keys()) == set(['Yes', 'No']):
        X_df.loc[:, col_name] = X_df[col_name].apply(lambda x: 1 if x == 'Yes' else 0)

In [33]:
real_col_names = ['Tenure', 'Monthly Charges', 'Total Charges']
scaler = StandardScaler()
real_cols = scaler.fit_transform(df.loc[:, real_col_names])
X_df[real_col_names] = real_cols

y = y.apply(lambda x: 1 if x == 'Yes' else 0)
X_df.fillna(value=0, inplace=True)

X_df = pd.get_dummies(X_df)
X = X_df.to_numpy(dtype=float)
y = y.to_numpy(dtype=float)

In [34]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [35]:
simpRandomForestClf = SimpleRandomForestClassifier(n_estimators=100, max_depth=20)

In [36]:
simpRandomForestClf.fit(X_train, y_train)

SimpleRandomForestClassifier(max_depth=20)

In [37]:
ratio = sum(y_test)/len(y_test)

In [38]:
y_pred_proba = simpRandomForestClf.predict_proba(X_test)
y_pred = simpRandomForestClf.predict(X_test, th=ratio)

In [39]:
prec = precision_score(y_test, y_pred) * 100
recall = recall_score(y_test, y_pred) * 100
f_score = f1_score(y_test, y_pred) * 100
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1]) * 100
print('Precision: {:.3f}%\n'.format(prec) + 
        'Recall: {:.3f}%\nF1 score: {:.3f}% \nROC-AUC score: {:.3f}%'.format(
        recall, f_score, roc_auc))

Precision: 48.084%
Recall: 80.000%
F1 score: 60.065% 
ROC-AUC score: 82.816%


In [42]:
randForest = RandomForestClassifier(n_estimators=100)

In [43]:
randForest.fit(X_train, y_train)

RandomForestClassifier()

In [44]:
y_pred_proba = randForest.predict_proba(X_test)
y_pred = randForest.predict(X_test)

In [45]:
prec = precision_score(y_test, y_pred) * 100
recall = recall_score(y_test, y_pred) * 100
f_score = f1_score(y_test, y_pred) * 100
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1]) * 100
print('Precision: {:.3f}%\n'.format(prec) + 
        'Recall: {:.3f}%\nF1 score: {:.3f}% \nROC-AUC score: {:.3f}%'.format(
        recall, f_score, roc_auc))

Precision: 60.498%
Recall: 49.275%
F1 score: 54.313% 
ROC-AUC score: 82.568%


# Часть 2. Градиентный бустинг

Во второй части лабораторной работы вы будете использовать датасет из прошлого "развлекательного" соревнования на Kaggle и (опционально) обучать модель и выполнять другие действия напрямую в kaggle kernels. 
В рамках соревнования была поставлена задача предсказания нормализованного от 0 до 1 места, которое занял игрок в матче в компьютерной игре PUBG на основании игровой статистики по этому матчу.




Для этого вам нужно:
1. Создать аккаунт на kaggle
2. Перейти в соревнование https://www.kaggle.com/c/pubg-finish-placement-prediction
3. Согласиться с правилами и получить к нему доступ
4. Нажать Late Submission и New Notebook.

В итоге у вас откроется нечто похожее на Jupyter Notebook в обертке Kaggle с доступом к датасету и возможностью использовать большинство популярных библиотек машинного обучения. Таким образом, не будет необходимости нагружать личный ноутбук датасетом, который в этот раз будет намного больше предыдущих.

Сессия автоматически оборвется через несколько часов, но вам времени хватит.


В конце нужно будет сгенерировать ответ в формате файла sample_submission_V2.csv и выполнить Late Submission следующими действиями:
1. В верхнем правом углу интерфейса ноутбука нажать Save Version
2. Там выбрать опцию Save & Commit и сохранить. Эта кнопка привет к тому, что весь ваш ноутбук будет пересчитан, а предсказания, которые вы сохраните в конце как preds.to_csv('submission.csv') можно будет отправить на оценку качества
3. После завершения пересчета ноутбука вы сможете открыть готовый файл и нажать там на вкладке output кнопку Submit.

В итоге вам нужно прислать мне (лектору) скриншот вашего лучшего сабмита и метрику качества модели **текстом**, чтобы я мог составить чисто на вашей группе таблицу. За относительные места вы получите **отдельную** оценку. 

Все вместе это звучит сложно, но в начале практики я покажу с помощью шаринга экрана, что нужно делать.


**Полезный совет**: В итоговом ноутбуке вам нет необходимости повторять все вычисления из задания (например, перебор гиперпараметров), а только повторить инженерию признаков и обучить лучшую модель. Он может быть отдельным от того ноутбука, в котором вы проведете EDA и непосредственно выполните лабу.


P.S. Описание датасета приведено на этой странице https://www.kaggle.com/c/pubg-finish-placement-prediction/data

## Задание 2.1 Работа с lightgbm 

In [ ]:
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder

Библиотека для градиентного бустинга lightgbm умеет работать с категориальными признаками без предварительного кодирования. Для этого их нужно только указать и преобразовать в числа. Сделать это можно с помощью OrdinalEncoder. В текущем датасете не так много категориальных переменных, поэтому это оказывается не принципиально (но в рамках курса вам это еще понадобится). Обратите внимание, что для тренировочного датасета у энкодера нужно вызывать fit_transform, а для тестового только transform. И ответьте мне на вопрос: почему?

In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)

С lgb удобно работать следующим образом.
1. Вы разбиваете ваши данные на тренировочные и валидационные и создаете с помощью lgb.Dataset два объекта dtrain и dval
2. Используете lgb.train, чтобы обучить модель. Все параметры обучения удобно хранить в одном словаре params
3. Предсказываете на тестовых данных с помощью полученной модели

Ниже показан близкий к реальностти псевдокод (обратите внимание, что можно напрямую разделить ваш финальный датафрейм и указывать переменными конкретный набор признаков. Так вам будет удобнее экспериментировать)

In [ ]:
df_train, df_val = train_test_split(df, ...)

In [ ]:
features = ['foo', 'bar', 'baz']
cagegorical_features = ['baz']
label = 'your_target_column'

In [ ]:
dtrain = lgb.Dataset(data=df_train[features],
                     label=df_train[label],
                     feature_name=features,
                     categorical_feature=categorical_feature)

dval = lgb.Dataset(data=df_val[features],
                   label=df_val[label],
                   feature_name=features,
                   categorical_feature=categorical_feature)

Ниже приведен пример списка параметров, который делает следующее
1. Устанавливает обычный тип бустинга (другие - это goss и dart)
2. Говорит использовать в качестве функции потерь и метрики mean absolute error (целевая метрика в этой задаче)
3. Ограничивает максимальное число листьев в дереве до 40
4. Отключает ограничение на максимальную глубину. Это один из приемов, введенный lightgbm, при которой мы можем ограничивать сложность дерева не глубиной, а максимальным количеством листьев. Однако, можно и явно ограничивать глубину. Попробовать не помешает.
5. Устанавливает скорость обучения 0.1. Рекомендуется подбирать параметры и проводить эксперименты именно с  такой скоростью или максимум скоростью 0.07-0.05, а для финального предсказания снизить ее до, например, 0.01
6. Устанавливает минимальное количество примеров для одного листа в 10. Параметр очень похож на тот, который вы уже использовали для деревьев и леса.
7. feature_fraction - для метода подпространств, bagging_fraction - для сэмплирования из исходного датасета (в этом случае без повторений)
8. Указывает 6 тредов (по количеству физических ядер). У вас этот параметр может быть другим для оптимальной работы
9. Устанавливает некоторые параметры обработки категориальных признаков. Предлагается вам разобраться с их назначением самостоятельно


Все параметры и другие советы по их тюнингу смотреть вот тут. https://lightgbm.readthedocs.io/en/latest/Parameters.html

Для достижения оптимального результата вам, вероятно, захочется использовать еще какие-то.

P.S. Набор гиперпараметров приведен для примера и упрощения вашей работы. Но это не значит, что значения гиперпараметров оптимальны, они местами взяты от балды.

In [ ]:
params = {
        'boosting_type': 'gbdt',
        'objective': 'mae',
        'metric': 'mae',
        'num_leaves': 40,
        'max_depth': None,
        'learning_rate': 0.1,
        'min_data_in_leaf': 10,
        'feature_fraction': 0.6,
        'bagging_fraction': 0.6,
        'bagging_freq': 1,
        #'bagging_freq': 5,
        'num_threads': 6,
    
        'cat_smooth': 10,
        'max_cat_threshold': 16,
        'max_cat_to_onehot': 4,
    }

Код ниже обучит модель на dtrain, произведет раннюю остановку по dval, используя params в качестве гиперпараметров. Всегда имеет смысл поставить очень много итераций бустинга, но контролировать обучение с помощью ранней остановки.

In [ ]:
model = lgb.train(params, dtrain, 
              # указываем валидационный датасет и тренировочный (хотим посмотреть качество и на нем тоже)
              # однако он будет проигнорирован механикой ранней остановки
              valid_sets=(dtrain, dval),
              # поставим очень большое количество итераций бустинга
              num_boost_round=10000,
              # но будем использовать раннюю остановку по качеству на валидационной выборке
              early_stopping_rounds=25,
              #будем выводить промежуточные результаты каждые 25 итераций
              verbose_eval=25)

Попробуйте добавить другие параметры, осмысленно поменять их значения (усилив или ослабив разные способы регуляризации и т.д.) перед тем как переходить к их подбору.

Полученную модель можно использовать для предсказания с помощью model.predict на новых данных. Обратите внимание, что передавать для предсказаний нужно не lgb.Dataset, а pd.DataFrame только с нужными столбцами, т.е. условно test_df[features]

## Задание 2.2. Особенности подбора гиперпараметров градиентного бустинга

В контексте подбора гиперпараметров в градиентом бустинге интересно проявляется функционал ранней остановки. 
С одной стороны есть LGBMRegressor, реализующий интерфейс, который можно использовать в Grid- или RandomizedSeach. Однако, в этом интерфейсе возможность ранней остановки есть только в методе fit. И оказывается, что учесть раннюю остановку в переборе гиперпараметров не получается. В итоге у вас каждый раз будет обучаться то количество деревьев, которое вы укажете в n_estimators. Это может привести к недообученным или переобученным деревьям, так как вы можете или указать слишком мало, или слишком много. 

Корректный перебор гиперпараметров возможен, если вы выбираете много num_boost_round и указываете early_stopping_rounds как в lgb.train выше. Попробуем это сделать одним из возможных способов

Для того, чтобы реализовать настройку гиперпараметров правильно вам нужно:
1. Использовать lgb.cv (интерфейс практически совпадает с lgb.cv), чтобы с помощью кросс-валидации и ранней остановки получать качество работы алгоритма на указанных params
2. Реализовать функцию eval_lgb, которая на вход принимает указанный список гиперпараметров, а на выходе возвращает значение метрики качества. ВНИМАНИЕ, некоторые параметры (objective, metric и т.д.) должны быть каждый раз одинаковы. Их передавать не нужно.
3. Использовать ParameterGrid или ParameterSampler и sklearn.model_selection для итерации по возможным наборам или указанному числу сэмплов (в зависисмости выбранного вами способа)
4. Вычислить значение метрики на выбранных вами комбинациях, отсортировать и получить лучшее
5. Обучить ваш алгоритм с помощью lgb.train на лучшем наборе параметров, попробовав дополнительно снизить скорость обучения

Ниже представлена заготовка функции eval_lgb. Не забудьте правильно указать параметры кросс-валидации и зафиксировать random_state!


In [ ]:
def eval_lgb(params):
    base_params = {
        'boosting_type': 'gbdt',
        'objective': 'mae',
        'metric': 'mae',
        'num_threads': 6
    }
    base_params.update(params)
    results = lgb.cv(...)
    ...

## Задание 2.3 (10) Байесовская оптимизация для подбора гиперпараметров

Байесовская оптимизация - это один из продвинутых способов перебора гиперпараметров модели, имеющий некоторые общие черты с рандомизированным поиском. В этом случае оптимизация состоит из двух частей: 
- exploration - исследование новых регионов поверхности гиперпараметров, на который происходит поиск оптимальных решений.
- exploitation - более подробный разбор регионов, для которых уже известно, что там качество лучше

Первая часть и есть наш случайный поиск, а вторая - использования баейсовского подхода к статистике ([вот тут крутой, но довольно сложный курс](https://www.coursera.org/learn/bayesian-methods-in-machine-learning)) для генерации новых комбинаций гиперпараметров, которые могут давать высокое качество. 

Для желающих подробно ознакомиться с темой оставляю следующию ссылку: https://distill.pub/2020/bayesian-optimization/

В рамках этого задания я вам предлагаю познакомиться с инструментом в утилитарном стиле. Вам предлагается провести оптимизацию гиперпараметров вашей модели с использованием библиотеки bayesian-optimization (conda install -c conda-forge bayesian-optimization) для lightgbm. https://github.com/fmfn/BayesianOptimization

Для этого вам нужно взять функцию eval_lgb и, возможно, адаптировать ее под новый вариант использования.
